In [1]:
data_root = '/cellar/users/hsher/ontoPrune/data/'
import pickle
from ddot import Ontology

# read pruned ontology
ont = Ontology.read_pickle(data_root+'prune_go_{:.2f}'.format(0.80))

In [2]:
ont

19626 genes, 2754 terms, 100290 gene-term relations, 3685 term-term relations
node_attributes: []
edge_attributes: ['Annotation Extension', 'Aspect', 'Assigned By', 'DB', 'DB Object ID', 'DB Object Name', 'DB Object Synonym', 'DB Object Type', 'DB:Reference', 'Date', 'Evidence Code', 'Gene Product Form ID', 'Qualifier', 'Taxon', 'With (or) From', 'biological_process', 'is_a']

In [3]:
ont.get_roots() # molecular function, cellular component, Biological process

['GO:0003674', 'GO:0005575', 'GO:0008150']

In [4]:
mf = ont.focus('GO:0003674')

Genes and Terms to keep: 18724


In [5]:
bp =ont.focus('GO:0008150')

Genes and Terms to keep: 19138


In [6]:
cc = ont.focus('GO:0005575')

Genes and Terms to keep: 19132


In [7]:
mf

17614 genes, 1110 terms, 30380 gene-term relations, 1151 term-term relations
node_attributes: []
edge_attributes: ['Annotation Extension', 'Aspect', 'Assigned By', 'DB', 'DB Object ID', 'DB Object Name', 'DB Object Synonym', 'DB Object Type', 'DB:Reference', 'Date', 'Evidence Code', 'Gene Product Form ID', 'Qualifier', 'Taxon', 'With (or) From', 'biological_process', 'is_a']

In [8]:
bp

17769 genes, 1369 terms, 47219 gene-term relations, 2238 term-term relations
node_attributes: []
edge_attributes: ['Annotation Extension', 'Aspect', 'Assigned By', 'DB', 'DB Object ID', 'DB Object Name', 'DB Object Synonym', 'DB Object Type', 'DB:Reference', 'Date', 'Evidence Code', 'Gene Product Form ID', 'Qualifier', 'Taxon', 'With (or) From', 'biological_process', 'is_a']

In [9]:
cc

18857 genes, 275 terms, 22691 gene-term relations, 296 term-term relations
node_attributes: []
edge_attributes: ['Annotation Extension', 'Aspect', 'Assigned By', 'DB', 'DB Object ID', 'DB Object Name', 'DB Object Synonym', 'DB Object Type', 'DB:Reference', 'Date', 'Evidence Code', 'Gene Product Form ID', 'Qualifier', 'Taxon', 'With (or) From', 'biological_process', 'is_a']

In [10]:
cc.to_table()

,Parent,Child,EdgeType
0,GO:0000439,GO:0000438,Child-Parent
1,GO:0005575,GO:0000110,Child-Parent
2,GO:0005575,GO:0000126,Child-Parent
3,GO:0005575,GO:0000127,Child-Parent
4,GO:0005575,GO:0000221,Child-Parent
...,...,...,...
22982,GO:0005575,hCG_1642624,Gene-Term
22983,GO:0005575,hCG_1984214,Gene-Term
22984,GO:0005761,hCG_1984214,Gene-Term
22985,GO:0015935,hCG_1984214,Gene-Term


In [11]:
import sys
sys.path.append('/cellar/users/hsher/ontoPrune')
from ontoPrune.prune import *
from ontoPrune.entropy import *

In [12]:
def write_to_dcell(ont, outfilename, remove_root_attached_genes = True):
    term_order = ont.topological_sorting(top_down = True, include_genes = False)
    
    ont_table = ont.to_table()
    
    if remove_root_attached_genes:
        to_be_remove = ont_table.loc[(ont_table['Parent'] == ont.get_roots()[0]) & (ont_table['EdgeType'] == 'Gene-Term')].index
        print('dropping {} gene-term'.format(len(to_be_remove)))
        ont_table.drop(index = to_be_remove, inplace = True)
        ont = Ontology.from_table(ont_table)
    ont_count = count_gene(ont)
    
    
    with open(outfilename, 'w') as f:
        for rt_term in term_order: # obey the topological order
            f.write('ROOT: {0} {1}\n'.format(rt_term, ont_count.loc[rt_term, 'indirect gene count']))
            gene_list, term_list = find_child(ont_table, rt_term)
            f.write('GENES: {}\n'.format(' '.join(gene_list)))
            f.write('TERMS: {}\n'.format(' '.join(term_list)))

In [13]:
write_to_dcell(cc, data_root+'cc_080.topo', remove_root_attached_genes = False)
write_to_dcell(bp, data_root+'bp_080.topo', remove_root_attached_genes = False)
write_to_dcell(mf, data_root+'mf_080.topo', remove_root_attached_genes = False)

/cellar/users/hsher/ontoPrune/ontoPrune/entropy.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  ont_count = pd.concat([dir_count, indir_count, term_count], axis = 1)


In [14]:
write_to_dcell(cc, data_root+'cc_080_drop.topo', remove_root_attached_genes = True)
write_to_dcell(bp, data_root+'bp_080_drop.topo', remove_root_attached_genes = True)
write_to_dcell(mf, data_root+'mf_080_drop.topo', remove_root_attached_genes = True)

dropping 18826 gene-term
dropping 17688 gene-term
dropping 17066 gene-term
